# Creating a New Column in Pandas Data Frame
***
The classic way of creating a new column in a Pandas data frame using other columns is through the use of the `.apply()` function. However, this is essentially a `for` loop that iterates through the entire data frame and therefore is not fast on large data frames. Can be described as not *pandorable*. <br>
This notebook is a short one that will create a dummy data frame, and perform the same operation but going through two styles:
- The classic `.apply()`
- A filter, `.reindex()` and join approach

And give the timing for each.

In [1]:
import pandas as pd

We will start with a small data set:

In [2]:
df = pd.read_csv('insurance_premiums.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 9 columns):
State                              51 non-null object
fatal_collisions                   51 non-null float64
fatal_collisions_speeding          51 non-null int64
fatal_collisions_alc               51 non-null int64
fatal_collisions_not_distracted    51 non-null int64
fatal_collisions_no_hist           51 non-null int64
premiums                           51 non-null float64
insurance_losses                   51 non-null float64
Region                             51 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 3.7+ KB


In [4]:
df.head()

,State,fatal_collisions,fatal_collisions_speeding,fatal_collisions_alc,fatal_collisions_not_distracted,fatal_collisions_no_hist,premiums,insurance_losses,Region
0,Alabama,18.8,39,30,96,80,784.55,145.08,South
1,Alaska,18.1,41,25,90,94,1053.48,133.93,West
2,Arizona,18.6,35,28,84,96,899.47,110.35,West
3,Arkansas,22.4,18,26,94,95,827.34,142.39,South
4,California,12.0,35,28,91,89,878.41,165.63,West


Now, say we want a new column that will be `1` if the premiums is over 1000, else `0`. Most common way is as follows:

In [5]:
apply_df = df.copy()

In [6]:
apply_df['expensive'] = apply_df.apply(lambda row: 1 
                                       if row['premiums'] > 1000
                                       else 0, axis=1)

In [7]:
apply_df[['State', 'premiums', 'expensive']].head()

,State,premiums,expensive
0,Alabama,784.55,0
1,Alaska,1053.48,1
2,Arizona,899.47,0
3,Arkansas,827.34,0
4,California,878.41,0


How fast did this run:

In [8]:
apply_df = df.copy()

In [9]:
%%timeit
apply_df['expensive'] = apply_df.apply(lambda row: 1 
                                       if row['premiums'] > 1000
                                       else 0, axis=1)

1.15 ms ± 61.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


This is a really small data frame, so pretty fast. But let's look at the other method:

In [10]:
join_df = df.copy()

In [11]:
# Filter
temp_df = join_df[join_df['premiums'] > 1000]

# Create new data frame of only the filtered items
temp_df = pd.DataFrame(data=[1]*len(temp_df),
                       index=temp_df.index,
                       columns=['expensive'])

# Reindex and join
join_df = join_df.join(temp_df.reindex(join_df.index, fill_value=0), how='inner')

In [12]:
join_df[['State', 'premiums', 'expensive']].head()

,State,premiums,expensive
0,Alabama,784.55,0
1,Alaska,1053.48,1
2,Arizona,899.47,0
3,Arkansas,827.34,0
4,California,878.41,0


Good! Same objective achieved, let's see how fast this method is:

In [13]:
join_df = df.copy()

In [14]:
%%timeit
temp_df = join_df[join_df['premiums'] > 1000]

temp_df = pd.DataFrame(data=[1]*len(temp_df),
                       index=temp_df.index,
                       columns=['expensive'])

new_df = join_df.join(temp_df.reindex(join_df.index, fill_value=0), how='inner')

1.73 ms ± 86.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


So, with small data sets the `.apply()` method is slightly faster. Let's make the data frame much larger (like most realistic data sets)

In [15]:
for i in range(10):
    df = df.append(df)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52224 entries, 0 to 50
Data columns (total 9 columns):
State                              52224 non-null object
fatal_collisions                   52224 non-null float64
fatal_collisions_speeding          52224 non-null int64
fatal_collisions_alc               52224 non-null int64
fatal_collisions_not_distracted    52224 non-null int64
fatal_collisions_no_hist           52224 non-null int64
premiums                           52224 non-null float64
insurance_losses                   52224 non-null float64
Region                             52224 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 4.0+ MB


In [18]:
# Make sure new index is unique for the join
df = df.reset_index().drop(columns=['index'])

Let's compare both methods again:

In [19]:
apply_df = df.copy()

In [20]:
%%timeit
apply_df['expensive'] = apply_df.apply(lambda row: 1 
                                       if row['premiums'] > 1000
                                       else 0, axis=1)

444 ms ± 9.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
join_df = df.copy()

In [22]:
%%timeit
temp_df = join_df[join_df['premiums'] > 1000]

temp_df = pd.DataFrame(data=[1]*len(temp_df),
                       index=temp_df.index,
                       columns=['expensive'])

new_df = join_df.join(temp_df.reindex(join_df.index, fill_value=0), how='inner')

6.41 ms ± 317 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As can be seen, with larger data frames (which is often the case, most data frames won't have 52 rows), the second method is much faster - and this will only get more extreme as the data frame grows.